In [1]:
!pip install rank_bm25 pypdf pdfplumber sentence-transformers faiss-cpu pandas transformers tabulate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
import re
import numpy as np
import pandas as pd
import pdfplumber
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss

2025-07-28 04:23:49.644755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753676629.824669      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753676629.880934      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# --- TEXT EXTRACTION & PROCESSING ---

def extract_clean_text(pdf_path):
    """
    Extracts and cleans text from financial PDF reports by:
    1. Extracting all page text
    2. Removing standard financial boilerplate
    3. Normalizing whitespace
    
    Args:
        pdf_path (str): Path to PDF file
        
    Returns:
        str: Cleaned text content
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            ptxt = page.extract_text()
            if ptxt:
                text += ptxt + "\n"
    
    # Remove standard financial report sections
    patterns = [
        r'Forward-Looking Statements.*',  # Legal disclaimers
        r'Contacts Investors:.*',         # Investor relations
        r'About Meta.*'                   # Corporate description
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.DOTALL|re.IGNORECASE)
    
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

def smart_chunk_text(text, chunk_size=100, overlap=20):
    """
    Segment text into coherent chunks preserving context through:
    - Sentence boundary awareness
    - Configurable chunk sizes 
    - Contextual overlap
    
    Args:
        text (str): Input document text
        chunk_size (int): Target tokens per chunk
        overlap (int): Tokens to overlap between chunks
        
    Returns:
        list: Context-preserving text chunks
    """
    sents = re.split(r'(?<=[.!?]) +', text)  # Splitting at sentence boundaries
    chunks, cur, tokens = [], [], 0
    
    for s in sents:
        s_tok = len(s.split())
        
        # Create new chunk if exceeding size
        if tokens + s_tok > chunk_size and cur:
            chunks.append(' '.join(cur))
            cur = cur[-overlap:] if overlap else []  # Maintain overlap
            tokens = sum(len(x.split()) for x in cur)
            
        cur.append(s)
        tokens += s_tok
    
    # Add final partial chunk
    if cur:
        chunks.append(' '.join(cur))
        
    return chunks

def get_embeddings(chunks, model_name="all-MiniLM-L6-v2"):
    """
    Generate dense vector embeddings for semantic search
    
    Args:
        chunks (list): Text chunks to embed
        model_name (str): SentenceTransformer model
        
    Returns:
        tuple: (embeddings array, embedding model)
    """
    embedder = SentenceTransformer(model_name)
    embs = embedder.encode(chunks, show_progress_bar=True).astype(np.float32)
    return embs, embedder

def build_faiss_index(embeddings):
    """
    Create FAISS index for efficient similarity search
    
    Args:
        embeddings (np.array): Array of text embeddings
        
    Returns:
        faiss.Index: Searchable vector index
    """
    idx = faiss.IndexFlatL2(embeddings.shape[1])  # Euclidean distance metric
    idx.add(embeddings)
    return idx

In [4]:
# --- ADVANCED RETRIEVAL SYSTEM ---

def hybrid_retrieve(query, embedder, idx, chunks, top_k=5):
    """
    Hybrid retrieval combining:
    1. Semantic search (FAISS)
    2. Lexical search (TF-IDF)
    
    Args:
        query (str): Search query
        embedder: SentenceTransformer instance
        idx: FAISS index
        chunks (list): Candidate chunks
        top_k (int): Results per method
        
    Returns:
        list: Unique combined results
    """
    # Semantic search
    q_sem = embedder.encode([query]).astype(np.float32)
    _, sem_I = idx.search(q_sem, top_k)
    sem_chunks = [chunks[i] for i in sem_I[0]]
    
    # Lexical search
    vectorizer = TfidfVectorizer().fit(chunks + [query])
    X = vectorizer.transform(chunks)
    q_X = vectorizer.transform([query])
    sims = (X * q_X.T).toarray().flatten()
    lex_order = sims.argsort()[-top_k:][::-1]
    lex_chunks = [chunks[i] for i in lex_order]
    
    # Merge with deduplication
    out, seen = [], set()
    for ch in sem_chunks + lex_chunks:
        if ch not in seen:
            out.append(ch)
            seen.add(ch)
    
    return out[:top_k]

def crossencoder_rerank(query, candidates, k=3):
    """
    Rerank candidates using cross-encoder for precision
    
    Args:
        query (str): Original question
        candidates (list): Retrieved chunks
        k (int): Final results to return
        
    Returns:
        list: Reordered candidates by relevance
    """
    cross = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[query, c] for c in candidates]
    scores = cross.predict(pairs)
    idxs = np.argsort(scores)[::-1]
    return [candidates[i] for i in idxs[:k]]

def context_limiter(chunks, max_chars=1700):
    """
    Truncates context to fit model limits while preserving relevance
    
    Args:
        chunks (list): Relevant chunks
        max_chars (int): Character limit
        
    Returns:
        str: Concatenated context
    """
    ctx = ""
    for ch in chunks:
        if len(ctx) + len(ch) <= max_chars:
            ctx += ch + "\n"
        else:
            break
    return ctx.strip()


In [5]:
# --- TABULAR DATA HAND`LING ---

def extract_tables(pdf_path):
    """
    Extract and validate tables from PDF with:
    - Robust error handling
    - Minimum structure requirements
    
    Args:
        pdf_path (str): PDF file path
        
    Returns:
        list: Valid DataFrames
    """
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for table in page.extract_tables():
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    # Minimum 2 columns and 1 data row
                    if df.shape[1] > 1 and df.shape[0] > 0:
                        tables.append(df)
                except Exception as e:
                    continue  # Skip malformed tables
    return tables

def match_tables(tables, query, top_k=2):
    """
    Match tables to queries using keyword scoring
    
    Args:
        tables (list): DataFrame candidates
        query (str): User question
        top_k (int): Max tables to return
        
    Returns:
        list: Relevant tables ordered by score
    """
    tokens = set(w.lower() for w in re.findall(r"\w+", query))
    score_idx = []
    
    for i, df in enumerate(tables):
        txt = df.to_string().lower()
        score = sum(txt.count(tok) for tok in tokens)
        score_idx.append((score, i))
    
    score_idx.sort(reverse=True)
    return [tables[i] for score, i in score_idx if score > 0][:top_k]

def format_tables_as_text(tables, maxlen=700):
    """
    Convert tables to readable strings with size constraints
    
    Args:
        tables (list): DataFrames to format
        maxlen (int): Max characters per table
        
    Returns:
        str: Formatted table text
    """
    return "\n\n".join([df.head().to_string()[:maxlen] for df in tables])

def query_needs_table(query):
    """
    Determine if question requires tabular data using keyword matching
    
    Args:
        query (str): User question
        
    Returns:
        bool: Whether to include tables
    """
    keywords = [
        "compare", "table", "net income", "operating expense",
        "trend", "margin", "difference", "summary",
        "balance sheet", "free cash flow", "revenue"
    ]
    return any(kw in query.lower() for kw in keywords)


In [6]:
# --- QUERY OPTIMIZATION ---

def optimize_query(query):
    """
    Simple query normalization:
    - Standardize financial terms
    - Expand abbreviations
    - Case normalization
    
    Args:
        query (str): Original question
        
    Returns:
        str: Normalized query
    """
    q = query.lower()
    q = q.replace("q1", "first quarter").replace("&", "and")
    # Future: Could use LLM for paraphrasing
    return q.capitalize() if q and (q[0].islower()) else query


In [7]:
# --- ANSWER GENERATION ---

def gen_answer(prompt):
    """
    Generate answers using TinyLlama 1.1B with:
    - Deterministic output
    - Financial response style
    - Context integration
    
    Args:
        prompt (str): Complete generation prompt
        
    Returns:
        str: Generated answer
    """
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    output = pipe(prompt, max_new_tokens=260, do_sample=False)[0]['generated_text']
    return output[len(prompt):].strip()

In [8]:
# --- EVALUATION METRICS ---

def precision_at_k(relevant, retrieved, k):
    """
    Calculate precision@k metric
    
    Args:
        relevant (list): Ground truth relevant items
        retrieved (list): System's retrieved items
        k (int): Evaluation depth
        
    Returns:
        float: Precision score
    """
    return len(set(relevant) & set(retrieved[:k])) / float(k) if k else 0

def recall_at_k(relevant, retrieved, k):
    """
    Calculate recall@k metric
    
    Args:
        relevant (list): Ground truth relevant items
        retrieved (list): System's retrieved items
        k (int): Evaluation depth
        
    Returns:
        float: Recall score
    """
    return len(set(relevant) & set(retrieved[:k])) / (len(set(relevant)) or 1)

In [9]:
# --- MAIN PIPELINE ---

def advanced_rag_pipeline(pdf_path, queries, ablate_rerank=False):
    """
    End-to-end advanced RAG pipeline:
    1. Document processing
    2. Hybrid retrieval + reranking
    3. Context-aware answering
    
    Args:
        pdf_path (str): Financial report PDF
        queries (list): Questions to answer
        ablate_rerank (bool): Whether to disable reranking
    """
    print("Extracting and chunking PDF...")
    text = extract_clean_text(pdf_path)
    chunks = smart_chunk_text(text)
    tables = extract_tables(pdf_path)
    print(f"Text Chunks: {len(chunks)}, Tables: {len(tables)}")
    
    # Setup search infrastructure
    embs, embedder = get_embeddings(chunks)
    idx = build_faiss_index(embs)

    for q in queries:
        print(f"\nQ: {q}")
        
        # Query processing
        opt_query = optimize_query(q)
        
        # Retrieval phase
        retrieved_chunks = hybrid_retrieve(opt_query, embedder, idx, chunks, top_k=5)
        if not ablate_rerank:
            retrieved_chunks = crossencoder_rerank(opt_query, retrieved_chunks, k=3)
        
        context = context_limiter(retrieved_chunks, max_chars=1700)
        
        # Table handling
        table_context = ""
        if query_needs_table(opt_query):
            matched = match_tables(tables, opt_query)
            if matched:
                table_context = format_tables_as_text(matched, 900)
        
        # Generation
        prompt = (
            f"You are a Meta financial analyst. Answer precisely using the context.\n"
            f"Text context:\n{context}\n"
            f"Table context:\n{table_context}\n"
            f"Question: {q}\nAnswer:"
        )
        
        answer = gen_answer(prompt)
        print(f"Answer:\n{answer}\n")


In [10]:
if __name__ == "__main__":
    # Example execution with comprehensive financial questions
    PDF = "/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf"
    queries = [
        "When did Meta report its first quarter 2024 results?",
        "What was Meta's total revenue for Q1 2024?",
        "How much did Meta's revenue increase compared to Q1 2023?",
        "What were the total costs and expenses for Meta in Q1 2024?",
        "How much income from operations did Meta report in Q1 2024?",
        "What was Meta's operating margin in Q1 2024?",
        "How much provision for income taxes did Meta have in Q1 2024?",
        "What was Meta's effective tax rate in Q1 2024?",
        "How much net income did Meta earn in Q1 2024?",
        "What was Meta's diluted earnings per share (EPS) for Q1 2024?",
        "What was the amount paid as dividends in Q1 2024?",
        "What revenue range does Meta expect for Q2 2024?",
        "What is the expected effect of foreign currency on Meta's revenue growth?",
    ]
    advanced_rag_pipeline(PDF, queries)

Extracting and chunking PDF...
Text Chunks: 29, Tables: 3


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Answer:
Meta expects revenue to be in the range of $36.45 billion to $36.35 billion for Q2 2024.


Q: What is the expected effect of foreign currency on Meta's revenue growth?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer:
The expected effect of foreign currency on Meta's revenue growth is to increase revenue by 6%.

